In [1]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_bc050f45180b4c1bb8886542b85a43fe = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='4DLJlzwg8MgMq4PD2ex7uQpqlXiuULzTTHhXPHtmCwyt',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_bc050f45180b4c1bb8886542b85a43fe.get_object(Bucket='merkantil-donotdelete-pr-8oggzmz2hvumtu',Key='Merkaintil_input.xlsx')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_0 = pd.read_excel(body)
df_data_0.head()

,woe_customer_type,woe_product_type,woe_pc,woe_age,woe_sales_channel,woe_nace_risk,DPD90_12m
0,-0.026821,-0.126048,-0.074399,0.583705,0.213637,0.691410,0
1,-0.026821,-0.126048,-0.191949,-1.130577,-0.911849,0.691410,0
2,-0.026821,-0.126048,0.684625,0.583705,0.213637,0.691410,0
3,-0.026821,-0.126048,0.684625,0.583705,0.213637,0.691410,0
4,-0.026821,-0.126048,-0.191949,-1.130577,-0.911849,-0.948982,0


In [2]:
df_data_0.shape

(10422, 7)

In [3]:
import numpy as np
X = np.asarray(df_data_0[['woe_customer_type', 'woe_product_type', 'woe_pc', 'woe_age', 'woe_sales_channel', 'woe_nace_risk']])
X[0:5]

array([[-0.02682054, -0.12604817, -0.0743993 ,  0.58370529,  0.21363652,
         0.6914096 ],
       [-0.02682054, -0.12604817, -0.19194917, -1.13057728, -0.91184899,
         0.6914096 ],
       [-0.02682054, -0.12604817,  0.68462526,  0.58370529,  0.21363652,
         0.6914096 ],
       [-0.02682054, -0.12604817,  0.68462526,  0.58370529,  0.21363652,
         0.6914096 ],
       [-0.02682054, -0.12604817, -0.19194917, -1.13057728, -0.91184899,
        -0.9489822 ]])

In [4]:
y = np.asarray(df_data_0['DPD90_12m'])
y [0:5]

array([0, 0, 0, 0, 0])

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=4)

In [42]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=3)
clf.fit(X_train, y_train)

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=3, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [43]:
yhat = clf.predict_proba(X_test)
yhat

array([[0.89188953, 0.10811047],
       [0.99009341, 0.00990659],
       [0.99009341, 0.00990659],
       ...,
       [0.97829269, 0.02170731],
       [0.98735415, 0.01264585],
       [0.97829269, 0.02170731]])

In [44]:
tech, y_predicted = zip(*yhat)
y_predicted

(0.10811047229758222,
 0.00990658599011966,
 0.00990658599011966,
 0.03669348526390549,
 0.03669348526390549,
 0.01146139084594712,
 0.02977686295376681,
 0.02939554217123126,
 0.05206369445685777,
 0.028173387872880196,
 0.021707308124039253,
 0.03850055798596734,
 0.028173387872880196,
 0.00990658599011966,
 0.013064865926833738,
 0.03509001018301887,
 0.03661467967041717,
 0.013064865926833738,
 0.028173387872880196,
 0.028173387872880196,
 0.013064865926833738,
 0.011042374556838694,
 0.015033473823951555,
 0.028173387872880196,
 0.013064865926833738,
 0.0453218055644967,
 0.00990658599011966,
 0.062061189874256975,
 0.02997419287777046,
 0.012645849637725312,
 0.015033473823951555,
 0.03661467967041717,
 0.03669348526390549,
 0.00990658599011966,
 0.012645849637725312,
 0.01644463418883941,
 0.01146139084594712,
 0.06431906400510351,
 0.040824610860518494,
 0.07012813972932082,
 0.03661467967041717,
 0.02977686295376681,
 0.013064865926833738,
 0.01146139084594712,
 0.012645849637

In [45]:
from sklearn.metrics import roc_auc_score
roc=roc_auc_score(y_test, y_predicted)
roc

0.7568053614863116

In [46]:
gini=2*roc-1
gini

0.5136107229726232

In [47]:
predictions_final = clf.predict_proba(X)
predictions_final

array([[0.98853861, 0.01146139],
       [0.95851171, 0.04148829],
       [0.99009341, 0.00990659],
       ...,
       [0.99676872, 0.00323128],
       [0.99288207, 0.00711793],
       [0.79976138, 0.20023862]])

In [48]:
tech1, y_predicted_all = zip(*predictions_final)
y_predicted_all

(0.01146139084594712,
 0.041488293912385066,
 0.00990658599011966,
 0.00990658599011966,
 0.10765836537362998,
 0.031337403891942,
 0.021707308124039253,
 0.00990658599011966,
 0.028173387872880196,
 0.028523550965160894,
 0.0047593518328829035,
 0.01146139084594712,
 0.031337403891942,
 0.03669348526390549,
 0.00990658599011966,
 0.019608650207901222,
 0.008099433598775069,
 0.013064865926833738,
 0.014889829333011955,
 0.03669348526390549,
 0.07045585267883488,
 0.021707308124039253,
 0.021707308124039253,
 0.0047593518328829035,
 0.04205346251545927,
 0.013064865926833738,
 0.013064865926833738,
 0.11428275934806813,
 0.021707308124039253,
 0.03669348526390549,
 0.028173387872880196,
 0.01644463418883941,
 0.04004794767945013,
 0.01146139084594712,
 0.031337403891942,
 0.028173387872880196,
 0.11398029004146257,
 0.013064865926833738,
 0.04004794767945013,
 0.021707308124039253,
 0.021707308124039253,
 0.033302675581628124,
 0.02977686295376681,
 0.02977686295376681,
 0.042053462515

In [49]:
roc2=roc_auc_score(y, y_predicted_all)
gini2=2*roc2-1
gini2

0.5326213026766722